## Define Slice Topology

### Imports 

In [ ]:
# Import libraries
%run ../setup/include/include_libraries.py
# load general variables
%run ../setup/config/load_variables.py
# load topology variables
%run topology_variables.ipynb

from state import *
import write_value

### Slice Topology

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # set whether you want the sites to be ptp capable
    ptp_capable= True
    
    if ptp_capable is True:
        print ("Generating PTP-capable random sites...")
        [site1,site2]= fablib.get_random_sites(count=2, filter_function=lambda x: x['ptp_capable'] is True and x['state'] == 'Active', avoid=['MAX', 'TACC', 'MASS', 'BRIST', 'TOKY', 'EDUKY', 'CERN', 'GATECH', 'SRI', 'CLEM'])
        print (f"Found {site1}, {site2}")
    else:
        if (node1_site =='' and node2_site ==''):
            print ("Generating random sites...")
            [site1,site2]  = fablib.get_random_sites(count=2)
            print (f"Found {site1}, {site2}")
    
    if (node1_site ==''):
        node1_site = site1
        write_value.write_value_to_file('topology_variables.ipynb', 'node1_site', node1_site)
    if (node2_site ==''):
        node2_site = site2
        write_value.write_value_to_file('topology_variables.ipynb', 'node2_site', node2_site)
    
    # Node1
    node1 = slice.add_node(name=node1_name, site=node1_site)
    iface1 = node1.add_component(model=node1_nic_model, name=node1_nic_name).get_interfaces()[0]

    # Node2
    node2 = slice.add_node(name=node2_name, site=node2_site)
    iface2  = node2.add_component(model=node2_nic_model, name=node2_nic_name).get_interfaces()[0]

    # NetworkS
    net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type=network1_type)
    net2 = slice.add_l3network(name=network2_name, interfaces=[iface2], type=network2_type)

    
    success = True
    
except Exception as e:
    print(f"Exception: {e}")
    success = False
    raise SystemExit("Stopping notebook execution due to exception")

### Set State to "SLICE_DEFINED" 

In [ ]:
if success is True:
    state_data = {'slice_name': slice_name, 'state': ['SLICE_DEFINED']}
    set_state_local(state_data)
else:
    print ('State is not updated due to error')